In [27]:
using LinearAlgebra
using Distributed
using NPZ
using JSON

In [2]:
# Add some workers for demonstration purposes
addprocs(4)

# Get the list of worker IDs
worker_ids = workers()

# Display the worker IDs
println("Current worker IDs: ", worker_ids)

# Count the number of workers
num_workers = length(worker_ids)
println("Number of workers running: ", num_workers)

Current worker IDs: [2, 3, 4, 5]
Number of workers running: 4


# Simple @everywhere examples

In [3]:
@everywhere begin
    # A simple function that squares its input
    square(x) = x^2
    
    # A variable for demonstration
    worker_id = myid()
end


In [4]:
# Apply the square function to numbers 1 through 10 in parallel
squared_numbers = pmap(square, 1:10)

println(squared_numbers)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [9]:
# Parallel sum of squares from 1 to 10
sum_of_squares = @distributed (*) for i = 1:30
    square(i)
end

println(sum_of_squares)

2562548187973812224


# Example of Quadruple For Loop

In [10]:
using Distributed

# Add workers if none are present
if nprocs() == 1  # Only the main process is running
    addprocs(4)  # Adjust this number based on your system's capabilities
end

@everywhere begin
    # Define the computation performed at each iteration
    function compute(i, j, k, l)
        # Placeholder for a more complex operation
        return i +j + k + l
    end
end


In [12]:
N = 100  # Define the size of the grid

# Accumulate results in a parallelized manner
results = @distributed (+) for i = 1:N
    local_sum = 0  # Initialize a local accumulator
    for j = 1:N
        for k = 1:N
            for l = 1:N
                local_sum += compute(i, j, k, l)
            end
        end
    end
    local_sum  # This value is returned and reduced across workers
end

# The variable `results` now holds the accumulated result of the computation
println("Accumulated result: ", results)

Accumulated result: 20200000000


# Import from Python

In [54]:
path = pwd()*"/four_site_data/"

spectra4 = JSON.parsefile(path*"spectra4.json")
a = spectra4[1]
b = spectra4[2]
c = spectra4[3]
d = spectra4[4]

# Load the JSON file
μ4_kets = JSON.parsefile(path*"μ4_kets.json")
μ4_bras = JSON.parsefile(path*"μ4_bras.json")
ν4_kets = JSON.parsefile(path*"ν4_kets.json")
ν4_bras = JSON.parsefile(path*"ν4_bras.json")
;

In [ ]:
using Distributed
addprocs(num_cores)  # Replace num_cores with the number of cores you want to use

@everywhere begin
  # Ensure all necessary functions, variables, and data are defined or loaded on all workers
  include("your_definitions_file.jl")  # If your functions and data are defined in a separate file
end

# If the result is a matrix, preallocate it. Adjust dimensions as necessary.
result_matrix = SharedArray{ComplexF64}(16, 16)  # Assuming a 16x16 result matrix for this example

@sync @distributed (+) for k3 = 1:length(d)
  local_sum = zeros(ComplexF64, 16, 16)  # Local accumulator for each worker
  for k2 = 1:length(c)
    for k1 = 1:length(b)
      for k0 = 1:length(a)
        # Your computation here, e.g.,
        contribution = W4_tensor[k0, k1, k2, k3] * (ν4_bras[k3]*μ4_kets[k2]) * (μ4_bras[k2]*ν4_kets[k1]) * (ν4_bras[k1]*μ4_kets[k0])
        local_sum += contribution  # Accumulate the local sum
      end
    end
  end
  @atomic result_matrix += local_sum  # Atomically add the local sum to the shared result matrix
end
